In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import lightgbm as lgb
root = "../"
pd.set_option('display.max_columns', 50)

# 2016

In [2]:
path = root + "data/raw/2_bancaditalia/2016/ind16_ascii/"

In [3]:
allb1_feat = [
    "nquest", # family id
    "nord", # person in family id
    'ylm', # Reddito complessivo netto annuo
    'partime', # part-time?
    'contratt',# type of contract?
    'tuttanno', # ha lavorato tutto l'anno?
    'mesilav', # se no, quanti mesi??
    'dimaz', # numero di persone stabilmente occupate, dipendenti in tutta Italia
    'oretot', # ore lavorative 
    'attest', # in italia? 
]

carcom_feat = [
    "nquest", # family id
    "nord", # person in family id
    'NCOMP', # numero di componenti della famiglia
    'NPERC', # numero di percettori della famiglia
    'NPERL', # numero di percettori di reddito da lavoro della famiglia
    'sex', # sesso
    'cit', # cittadinanza italiana
    'studio', # titolo studio
    'tipolau', # tipo laurea
    'votoedu', # voto: x punti 
    'suedu', # su y punti
    'selode', # con lode?
    'annoedu', # anno laurea
    'tipodip', # tipo diploma
    'apqual', # * vedi analysis
    'nace', # ** vedi analysis
    'ireg',  # ISTAT code regione
    'ETA', # età in anni
    'isco', # classificazione ISCO delle professioni
    'CLETA5', # classe di età
    'QUAL', # status del lavoratore
    'SETT', # settore di attività  
    'AREA3', # area (less)
    'AREA5', # area 
    'ACOM4C', # ampiezza comune (less?)
    'ACOM5', # ampiezza comune 
]


lavoro_feat = [ "nquest", # family id
                "nord", # person in family id
                "nesplav", # n. esp. lav
                "acontrib", # anni contrib.
                "etalav", # età cominciato lav.
                "etalav2", # età cominciato lavoro corrente
                ]

names_map = {'nquest' : 'id_famiglia',
            'nord' : 'id_persona',
            'ylm' : 'reddito_netto',
            'attivp' : 'attivita_princ',
            'contratt' : 'contratto',
            'attest' : 'in_italia',
            'dimaz' : 'dimensioni_azienda',
            'oretot' : 'ore_settimana',
            'cit' : 'cit_italiana',
            'studio' : 'titolo_studio',
            'tipolau' : 'tipo_laurea',
            'tipodip' : 'tipo_diploma',
            'apqual' : 'qualifica',
            'nace' : 'settore',
            'ireg' : 'regione',
            'isco' : 'codice_lavoro_isco',
            'CLETA5' : 'eta_discreta_5',
            'QUAL' : 'status_lavoratore',
            'SETT' : 'settore_less',
            'AREA3' : 'zona_less',
            'AREA5' : 'zona',
            'ACOM4C' : 'ampiezza_comune_less',
            'ACOM5' : 'ampiezza_comune',
            'NCOMP' : 'n_comp_famiglia',
            'NPERC' : 'n_percett_famiglia', 
            'NPERL' : 'n_lavor_famiglia', 
            "nesplav" : "n_esp_lavorative",
            "acontrib" : "anni_contributi", 
            "etalav" : "eta_primo_lavoro",
            "etalav2" : "eta_lavoro_corrente",
            }

In [4]:
def preprocess_2016(path):
    # load and merge
    allb1 = pd.read_csv(path + 'allb1.csv')[allb1_feat]
    carcom = pd.read_csv(path + "carcom16.csv")[carcom_feat]
    lavoro = pd.read_csv(path + 'lavoro.csv')[lavoro_feat]
    keys = ["nquest","nord"]
    df = pd.merge(allb1,carcom,on=keys,how="left")
    df = pd.merge(df,lavoro,on=keys,how="left")

    # change naming
    df = df.rename(columns = names_map)

    # add perc voto
    df["voto_perc"] = df["votoedu"] / df["suedu"]
    df = df.drop(columns = ["votoedu","suedu"])

    # reddito dipendente positivo
    df = df[df.reddito_netto>0].reset_index(drop=True)

    # lavoro in Italia
    df = df.loc[df.in_italia == 1]
    df = df.drop(columns = "in_italia")

    # lavoro tutto l'anno
    df = df.loc[df.tuttanno == 1]
    df = df.drop(columns = "tuttanno")
    df = df.drop(columns = "mesilav")

    # minors
    df = df.drop(columns = ["id_famiglia","id_persona"])
    
    df = df.reset_index(drop=True)
    
    
    return df

df = preprocess_2016(path)
if not os.path.isdir(root + "data/preprocessed/bancaditalia/"):
    os.makedirs(root + "data/preprocessed/bancaditalia/")
df.to_csv(root + "data/preprocessed/bancaditalia/2016.csv",index=False)

In [5]:
print(df.shape)
df.head()

(3970, 32)


,reddito_netto,partime,contratto,dimensioni_azienda,ore_settimana,n_comp_famiglia,n_percett_famiglia,n_lavor_famiglia,sex,cit_italiana,titolo_studio,tipo_laurea,selode,annoedu,tipo_diploma,qualifica,settore,regione,ETA,codice_lavoro_isco,eta_discreta_5,status_lavoratore,settore_less,zona_less,zona,ampiezza_comune_less,ampiezza_comune,n_esp_lavorative,anni_contributi,eta_primo_lavoro,eta_lavoro_corrente,voto_perc
0,10000.0,1,1.0,1,30,5,4,4,2,1,2,NaN,NaN,NaN,NaN,1,17.0,8,58,32.0,4,1,3,1,2,2,4,3.0,24.0,16.0,40.0,NaN
1,9000.0,1,1.0,2,35,5,4,4,2,1,5,NaN,0.0,1998.0,2.0,1,17.0,8,37,91.0,2,1,3,1,2,2,4,2.0,14.0,19.0,25.0,0.680000
2,8000.0,1,1.0,1,20,5,4,4,2,1,3,NaN,NaN,NaN,NaN,1,9.0,8,25,51.0,1,1,4,1,2,2,4,1.0,6.0,18.0,NaN,NaN
3,16000.0,1,1.0,2,40,5,4,4,1,1,3,NaN,NaN,NaN,NaN,1,3.0,8,35,72.0,2,1,2,1,2,2,4,3.0,12.0,17.0,33.0,NaN
4,10000.0,2,2.0,1,20,2,2,1,2,1,5,NaN,0.0,1971.0,1.0,2,17.0,9,63,53.0,4,2,3,2,3,0,2,1.0,NaN,20.0,NaN,0.833333
